# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 52 new papers today
          16 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.07441


extracting tarball to tmp_2212.07441...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.07447


extracting tarball to tmp_2212.07447...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.08015


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.08007/reftest4-2.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.08015...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'mchardy_4395_clean.bbl' from 'tmp_2212.08015/mchardy_4395_clean.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07441-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07441) | **The accretion history of the Milky Way. I. How it shapes globular  clusters and dwarf galaxies**  |
|| Francois Hammer, et al. -- incl., <mark>Hefan Li</mark> |
|*Appeared on*| *2022-12-16*|
|*Comments*| *18 pages, 8 Figures, 2 Tables with orbital properties of globular clusters, accepted in MNRAS, December the 12*|
|**Abstract**| Halo inhabitants are individual stars, stellar streams, star and globular clusters, and dwarf galaxies. Here we compare the two last categories that include objects of similar stellar mass, which are often studied as self-dynamical equilibrium systems. We discover that the half-light radius of globular clusters depends on their orbital pericenter and total energy, and that Milky Way (MW) tides may explain the observed correlation. We also suggest that the accretion epoch of stellar systems in the MW halo can be calibrated by the total orbital energy, and that such a relation is due to both the mass growth of the MW and dynamical friction affecting mostly satellites with numerous orbits. This calibration starts from the bulge, to Kraken, Gaia Sausage Enceladus, Sagittarius stellar systems, and finally to the new coming dwarfs, either or not linked to the vast-polar structure. The most eccentric globular clusters and dwarfs have their half-light radius scaling as the inverse of their binding energy, and this over more than two decades. This means that earlier arriving satellites are smaller due to the tidal effects of the MW. Therefore, most halo inhabitants appear to have their structural parameters shaped by MW tides and also by ram-pressure for the most recent arrivals, the dwarf galaxies. The correlations found in this study can be used as tools to further investigate the origin of globular clusters and dwarfs, as well as the assembly history of our Galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07447-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07447) | **Simulations of star forming main sequence galaxies in Milgromian gravity**  |
|| Srikanth T. Nagesh, et al. -- incl., <mark>Mahmood Roshan</mark>, <mark>Hongsheng Zhao</mark> |
|*Appeared on*| *2022-12-16*|
|*Comments*| *22 pages, 23 figures. Accepted for publication by the Monthly Notices of Royal Astronomical Society (MNRAS)*|
|**Abstract**| We conduct hydrodynamical MOND simulations of isolated disc galaxies over the stellar mass range $M_{\star}/M_\odot = 10^7 - 10^{11}$ using the adaptive mesh refinement code \textsc{phantom of ramses} (\textsc{por}), an adaptation of the \textsc{ramses} code with a Milgromian gravity solver. The scale lengths and gas fractions are based on observed galaxies, and the simulations are run for 5~Gyr. The main aim is to see whether existing sub-grid physics prescriptions for star formation and stellar feedback reproduce the observed main sequence and reasonably match the Kennicutt-Schmidt relation that captures how the local and global star formation rates relate to other properties. Star formation in the models starts soon after initialisation and continues as the models evolve. The initialized galaxies indeed evolve to a state which is on the observed main sequence, and reasonably matches the Kennicutt-Schmidt relation. The available formulation of sub-grid physics is therefore adequate and leads to galaxies that largely behave like observed galaxies, grow in radius, and have flat rotation curves $-$ provided we use Milgromian gravitation. Furthermore, the strength of the bars tends to be inversely correlated with the stellar mass of the galaxy, whereas the bar length strongly correlates with the stellar mass. Irrespective of the mass, the bar pattern speed stays constant with time, indicating that dynamical friction does not affect the bar dynamics. The models demonstrate Renzo's rule and form structures at large radii, much as in real galaxies. In this framework, baryonic physics is thus sufficiently understood to not pose major uncertainties in our modelling of global galaxy properties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07449-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07449) | **A Measurement of the Cosmic Optical Background and Diffuse Galactic  Light Scaling from the R < 50 AU New Horizons-LORRI Data**  |
|| Teresa Symons, et al. -- incl., <mark>Carey Lisse</mark> |
|*Appeared on*| *2022-12-16*|
|*Comments*| *36 pages, 22 figures, 8 tables; accepted for publication in ApJ*|
|**Abstract**| Direct photometric measurements of the cosmic optical background (COB) provide an important point of comparison to both other measurement methodologies and models of cosmic structure formation, and permit a cosmic consistency test with the potential to reveal additional diffuse sources of emission. The COB has been challenging to measure from Earth due to the difficulty of isolating it from the diffuse light scattered from interplanetary dust in our solar system. We present a measurement of the COB using data taken by the Long-Range Reconnaissance Imager (LORRI) on NASA's New Horizons mission, considering all data acquired to 47 AU. We employ a blind methodology where our analysis choices are developed against a subset of the full data set, which is then unblinded. Dark current and other instrumental systematics are accounted for, including a number of sources of scattered light. We fully characterize and remove structured and diffuse astrophysical foregrounds including bright stars, the integrated starlight from faint unresolved sources, and diffuse galactic light. For the full data set, we find the surface brightness of the COB to be $\lambda I_{\lambda}^{\mathrm{COB}}$ $=$ 21.98 $\pm$ 1.23 (stat.) $\pm$ 1.36 (cal.) nW m$^{-2}$ sr$^{-1}$. This result supports recent determinations that find a factor of 2 ${-}$ 3 $\times$ more light than expected from the integrated light from galaxies and motivate new diffuse intensity measurements with more capable instruments that can support spectral measurements over the optical and near-IR. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07881-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07881) | **Identifying AGN host galaxies with convolutional neural networks**  |
|| <mark>Ziting Guo</mark>, John F. Wu, Chelsea E. Sharon |
|*Appeared on*| *2022-12-16*|
|*Comments*| *6 pages, 2 figures. Accepted to the 2022 NeurIPS conference ML4PS workshop*|
|**Abstract**| Active galactic nuclei (AGN) are supermassive black holes with luminous accretion disks found in some galaxies, and are thought to play an important role in galaxy evolution. However, traditional optical spectroscopy for identifying AGN requires time-intensive observations. We train a convolutional neural network (CNN) to distinguish AGN host galaxies from non-active galaxies using a sample of 210,000 Sloan Digital Sky Survey galaxies. We evaluate the CNN on 33,000 galaxies that are spectrally classified as composites, and find correlations between galaxy appearances and their CNN classifications, which hint at evolutionary processes that affect both galaxy morphology and AGN activity. With the advent of the Vera C. Rubin Observatory, Nancy Grace Roman Space Telescope, and other wide-field imaging telescopes, deep learning methods will be instrumental for quickly and reliably shortlisting AGN samples for future analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07908-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07908) | **25,000 optical fiber positioning robots for next-generation cosmology**  |
|| <mark>Joseph H. Silber</mark>, et al. |
|*Appeared on*| *2022-12-16*|
|*Comments*| *6 pages, 8 figures, presented at conference Thirty-Seventh Annual Meeting of The American Society for Precision Engineering, 2022-10-14*|
|**Abstract**| Massively parallel multi-object spectrographs are on the leading edge of cosmology instrumentation. The highly successful Dark Energy Spectroscopic Instrument (DESI) which begun survey operations in May 2021, for example, has 5,000 robotically-actuated multimode fibers, which deliver light from thousands of individual galaxies and quasars simultaneously to an array of high-resolution spectrographs off-telescope. The redshifts are individually measured, thus providing 3D maps of the Universe in unprecedented detail, and enabling precise measurement of dark energy expansion and other key cosmological parameters. Here we present new work in the design and prototyping of the next generation of fiber-positioning robots. At 6.2 mm center-to-center pitch, with 1-2 um positioning precision, and in a scalable form factor, these devices will enable the next generation of cosmology instruments, scaling up to instruments with 10,000 to 25,000 fiber robots. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.07978-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.07978) | **Hubble Space Telescope survey of Magellanic Cloud star clusters.  Photometry and astrometry of 113 clusters and early results**  |
|| A. P. Milone, et al. -- incl., <mark>N. Langer</mark>, <mark>M. Libralato</mark>, <mark>C. Li</mark> |
|*Appeared on*| *2022-12-16*|
|*Comments*| *37 pages, 27 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| In the past years, we have undertaken an extensive investigation of LMC and SMC star clusters based on HST data. We present photometry and astrometry of stars in 101 fields observed with the WFC/ACS, UVIS/WFC3 and NIR/WFC3 cameras. These fields comprise 113 star clusters. We provide differential-reddening maps and illustrate various scientific outcomes that arise from the early inspection of the photometric catalogs. In particular, we provide new insights on the extended main-sequence turn-off (eMSTO) phenomenon: i) We detected eMSTOs in two clusters, KMHK361 and NGC265, which had no previous evidence of multiple populations. This finding corroborates the conclusion that the eMSTO is a widespread phenomenon among clusters younger than ~2 Gyr. ii) The homogeneous color-magnitude diagrams (CMDs) of 19 LMC clusters reveal that the distribution of stars along the eMSTO depends on cluster age. iii) We discovered a new feature along the eMSTO of NGC1783, which consists of a distinct group of stars going on the red side of the eMSTO in CMDs composed of ultraviolet filters. Furthermore, we derived the proper motions of stars in the fields of view of clusters with multi-epoch images. Proper motions allowed us to separate the bulk of bright field stars from cluster members and investigate the internal kinematics of stellar populations in various LMC and SMC fields. As an example, we analyze the field around NGC346 to disentangle the motions of its stellar populations, including NGC364 and BS90, young and pre-MS stars in the star-forming region associated with NGC346, and young and old field stellar populations of the SMC. Based on these results and the fields around five additional clusters, we find that young SMC stars exhibit elongated proper-motion distributions that point toward the LMC, thus bringing new evidence for a kinematic connection between the LMC and SMC. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08007-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08007) | **Characterising cosmic birefringence in the presence of galactic  foregrounds and instrumental systematic effects**  |
|| <mark>Baptiste Jost</mark>, Josquin Errard, Radek Stompor |
|*Appeared on*| *2022-12-16*|
|*Comments*| *26 pages, 9 figures*|
|**Abstract**| We study a possibility of constraining the isotropic cosmic birefringence with help of cosmic microwave background polarisation data without relying on any assumptions about the Galactic foreground angular power spectra and in particular on their EB correlation. We propose a new analysis framework aiming at measuring isotropic cosmic birefringence, based on a generalised parametric component separation approach, which accounts simultaneously on the presence of galactic foregrounds, relevant instrumental effects and external priors. We find that in the context of an upcoming multi-frequency CMB instrument, assuming in-lab calibration priors we are able to constrain instrumental polarisation angle for each frequency band and correct the observed sky component maps accordingly. We then produce an instrumental-effect-corrected and foreground-cleaned CMB map, which we use to estimate the isotropic birefringence angle and the tensor-to-scalar ratio, accounting on statistical and systematic uncertainties incurred during the entire procedure. In particular, in the case of a Simons Observatory-like, three Small Aperture Telescopes, we derive a uncertainty on the birefringence angle of $\sigma(\beta_{b}) = 0.07^\circ$, assuming calibration priors for all frequency channels with the precision of $\sigma_{\alpha_i}= 0.1^\circ$ and the standard cosmology. This implies that, using our method and given the calibration precision expected for current, the near future ground-based multi-frequency experiments could confirm or disprove the recently detected value of $\beta_b=0.35^\circ$ with a significance of up to $5 \sigma$. [abridged version] |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.08015-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.08015) | **First detection of the outer edge of an AGN accretion disc: Very fast  multiband optical variability of NGC 4395 with GTC/HiPERCAM and LT/IO:O**  |
|| I. M. McHardy, et al. -- incl., <mark>S. P. Littlefair</mark> |
|*Appeared on*| *2022-12-16*|
|*Comments*| *19 pages, 27 figures*|
|**Abstract**| We present fast (~200s sampling) ugriz photometry of the low mass AGN NGC 4395 with the Liverpool Telescope, followed by very fast (3s sampling) us, gs, rs, is and zs simultaneous monitoring with HiPERCAM on the 10.4m GTC. These observations provide the fastest ever AGN multiband photometry and very precise lag measurements. Unlike in all other AGN, gs lags us by a large amount, consistent with disc reprocessing but not with reprocessing in the Broad Line Region (BLR). There is very little increase in lag with wavelength at long wavelengths, indicating an outer edge (Rout) to the reprocessor. We have compared truncated disc reprocessing models to the combined HiPERCAM and previous X-ray/UV lags. For the normally accepted mass of 3.6E5 solar, we obtain reasonable agreement with zero spin, Rout ~1700 Rg, and the DONE physically-motivated temperature-dependent disc colour correction factor (fcol). A smaller mass of 4E4 solar can only be accomodated if fcol=2.4, which is probably unrealistically high. Disc self gravity is probably unimportant in this low mass AGN but an obscuring wind may provide an edge. For the small mass the dust sublimation radius is similar to Rout, so the wind could be dusty. However for the more likely large mass the sublimation radius is further out so the optically-thick base of a line-driven gaseous wind is more likely. The inner edge of the BLR is close to Rout in both cases. These observations provide the first good evidence for a truncated AGN disc and caution that truncation should be included in reverberation lag modelling. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

133  publications in the last 7 days.
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers